# Comparing Magnetopause Models to Spacecraft Observations with PyHC Packages

*Authors: Shawn Polson, Rebecca Ringuette, Lutz Rastaetter, Eric Grimes, Jonathan Niehof, Nicholas Murphy, Yihua Zheng*

https://agu.confex.com/agu/fm21/meetingapp.cgi/Paper/911907

## Table of contents

 * [Abstract](https://deepnote.com/project/PyHC-Paper-EBuWRj_QSXikjqTz5wigrA/%2FPyHC%20Science%20Paper-AGU.ipynb#00002-ec7743d6-f371-423d-9ce1-df33ca809d2b)
 * [Introduction](https://deepnote.com/project/PyHC-Paper-EBuWRj_QSXikjqTz5wigrA/%2FPyHC%20Science%20Paper-AGU.ipynb#00004-44154988-f663-48b1-b200-bdc46ecb7d85)
 * [Get MMS data from pySPEDAS](https://deepnote.com/project/PyHC-Paper-EBuWRj_QSXikjqTz5wigrA/%2FPyHC%20Science%20Paper-AGU.ipynb#00005-f68ecab7-9f53-4519-a036-b834b98718f8)
 * [Use the MMS data with SpacePy](https://deepnote.com/project/PyHC-Paper-EBuWRj_QSXikjqTz5wigrA/%2FPyHC%20Science%20Paper-AGU.ipynb#00009-e7193fd7-5d2b-445e-a046-f1cc04308fb5)
 * [Prepare data for PlasmaPy](https://deepnote.com/project/PyHC-Paper-EBuWRj_QSXikjqTz5wigrA/%2FPyHC%20Science%20Paper-AGU.ipynb#00018-e120b696-be5c-4a3e-a6e5-541ef471d42e)
 * [Calculate plasma parameters](https://deepnote.com/project/PyHC-Paper-EBuWRj_QSXikjqTz5wigrA/%2FPyHC%20Science%20Paper-AGU.ipynb#00029-94de09d9-9e7d-4cd0-b238-be9e84fc719a)
 * [Plot the plasma parameters and compare](https://deepnote.com/project/PyHC-Paper-EBuWRj_QSXikjqTz5wigrA/%2FPyHC%20Science%20Paper-AGU.ipynb#00054-1440f5e0-cb62-40d5-bccb-e40ffa4334de)
 * [Conclusion](https://deepnote.com/project/PyHC-Paper-EBuWRj_QSXikjqTz5wigrA/%2FPyHC%20Science%20Paper-AGU.ipynb#00061-340d7ce5-b6ba-4fff-943a-9269cc53212a)

## Abstract

Here we demonstrate an adaptable Python workflow to detect magnetopause locations. We compare&nbsp;these locations to magnetopause models using real-world observations&nbsp;and show how simple it is to do using our software packages. We use pySPEDAS to get MMS data, we get models of the magnetopause and plot them with the MMS data using SpacePy, and we calculate plasma parameters from the MMS data with PlasmaPy to augment our detections of the crossings. Then we compare the locations of the detected crossings with those estimated by the model to test their accuracy.

## Introduction

The Python in Heliophysics Community (PyHC) is a community that promotes and facilitates the use of Python for heliophysics. We host a number of useful software packages and showcase a few of them here. Our mission is to facilitate scientific discovery by promoting the use and development of sustainable open-source Python software across the solar and space physics community; to improve communication and collaboration between disciplines, developers, and users; to establish and maintain development standards; and to foster interoperability and reproducibility.

In this paper, we compare magnetopause models to spacecraft observations to test the accuracy of the models. We use five PyHC packages to do this: pySPEDAS, SpacePy, PlasmaPy, PyTplot, and Kamodo. The magnetopause is the outer limit of Earth's magnetic field. Models like Shue or OpenGGCM estimate where the magnetopause is. NASA's Magnetospheric Multiscale Mission (MMS) mission orbits four spacecraft around Earth that regularly cross the magnetopause boundary. We detect where these crossings happen using functionalities from our PyHC packages and compare the locations to the models' estimates.

## Get MMS data from pySPEDAS:

### Imports

First, we need to import everything. We import `pyspedas` to load the data and `tplot` from `pytplot` to create plots. Then we impo the relevant `spacepy` packages and set a couple environment variables. Finally, we make sure we have the solar wind data updated in our `.spacepy/` directory.

In [ ]:
# # pySPEDAS imports
# !pip install --upgrade https://github.com/MAVENSDC/PyTplot/archive/matplotlib-backend.zip
# !pip install pyspedas==1.2.2 
import pyspedas
from pytplot import tplot
import pytplot

# Imports of relevant SpacePy packages
import spacepy.coordinates
import spacepy.empiricals
import spacepy.time
import spacepy.toolbox
import numpy

# # Set CDF environment variables since they can't be set during initialization
# import os
# os.environ["CDF_BASE"] = "/root/work/dependencies/cdf38_0-dist"
# os.environ["CDF_LIB"] = "/root/work/dependencies/cdf38_0-dist/lib"

# # Make sure the solar wind data are up-to-date
# !cp -R /root/work/dependencies/spacepy/spacepy/data /root/.spacepy/  # Manually copy the data from persistent storage to avoid spacepy.toolbox.update() call

### Set the time range for loading the data.

The first time range (on Oct 16, 2015) is the time range from the Science paper:

J. L. Burch et al, Electron-scale measurements of magnetic reconnection in space (2016)

The next 4 time ranges were chosen from the MMS Scientist in the Loop (SITL) events database where the description mentioned observations of a full magnetopause crossing. There were over 300 such events in the database, and the following were mostly chosen at random, but I tried to only include time ranges where the crossings were fairly obvious in the FGM and plasma data. 

Most of these events show multiple magnetopause crossings over the time interval.

In [ ]:
tranges = [
    ['2015-10-16/13:00', '2015-10-16/13:10'], # time range from J. L. Burch et al, Electron-scale measurements of magnetic reconnection in space (2016)
    ['2018-03-06/20:20', '2018-03-06/20:30'],
    ['2019-02-08/13:20', '2019-02-08/13:40'],
    ['2020-11-24/21:30', '2020-11-24/21:50'],
    # ['2017-03-31/23:00', '2017-04-01/23:00'], # Jon's fantastic time range from his first notebook
    ['2019-02-09', '2019-02-09/01:00']        # Definitely a crossing in this range
]

trange = tranges[0]

### Load the MEC, FGM and DIS data

#### Why the specific instrument/variable was chosen:
- MEC data were chosen to retrieve the spacecraft location at the times of the magnetopause crossings
- FGM and DIS data were chosen to show the magnetic field and ion population change across the magnetopause boundary

#### Why the data level was chosen:
- For all cases, we use the science-quality, level 2 fast survey data for probe 1.

#### MMS Ephemeris and Coordinates (MEC) data
- Data can be loaded with `pyspedas` by calling `pyspedas.mission.instrument()`, with the options set via keyword arguments, e.g., MEC data can be loaded by calling `pyspedas.mms.mec()`.
- These data show the spacecraft location at the times of the magnetopause crossings

#### Fluxgate Magnetometer (FGM) data
- These data show the magnetic field as the spacecraft crosses the magnetopause

#### Now load the ion moments data from the Fast Plasma Investigation (FPI)
- These data show how the ion population changes as the spacecraft crosses the magnetopause


In [ ]:
def load_mms_data(time_range):
    """Loads MMS MEC, FGM, and FPI data."""
    mec_vars = pyspedas.mms.mec(trange=time_range, time_clip=True, probe=1, level='l2', data_rate='srvy', varformat='*_r_gsm')  # Cartesian GSM coords
    fgm_vars = pyspedas.mms.fgm(trange=time_range, time_clip=True, probe=1, level='l2', data_rate='srvy')
    ion_vars = pyspedas.mms.fpi(trange=time_range, datatype=['dis-moms', 'des-moms'], level='l2', data_rate='fast', time_clip=True, center_measurement=True)

    return mec_vars, fgm_vars, ion_vars

## Use the MMS data with SpacePy:

SpacePy has a simple way to calculate position in GSE as a function of local time in the equatorial plane, and exposes more flexible uses of the model for more involved needs, which we use here. The Shue model of the magnetopause is used.

In [ ]:
def spacecraft_magnetopause_calculations(mms_mec_vars):
    """Returns epoch, 
               pos,
               distance between spacecraft and magnetopause, 
               magnetopause's distance from Earth, 
               spacecraft's distance from Earth, 
               and solar zenith angle.
    """
    data = pytplot.get_data(mms_mec_vars[0])
    pos_gsm = data.y
    ticks = spacepy.time.Ticktock(data.times, dtype='UNX')
    epoch = ticks.UTC
    c = spacepy.coordinates.Coords(pos_gsm, 'GSM', 'car', units='km', ticks=ticks)
    pos = c.convert('GSE', 'car').data

    # Get the Shue coefficients
    alpha = []  # Shue flaring angle
    standoff = spacepy.empiricals.getMPstandoff(ticks, alpha=alpha)  # Shue subsolar standoff
    alpha = numpy.array(alpha)

    # Solar zenith angle of s/c position (angle with GSE +x)
    sza = numpy.arctan2(pos[:, 0], numpy.sqrt((pos[:, :2] ** 2).sum(axis=1)))
    # Radial distance to MP along Earth-SC line (application of Shue coefficients)
    mp_dist = standoff * (2. / (1 + numpy.cos(sza))) ** alpha
    # Radial distance to SC
    sc_dist = numpy.sqrt((pos ** 2).sum(axis=1)) / 6378
    # How far is SC outside of MP?
    sc_to_mp = sc_dist - mp_dist

    return epoch, pos, sc_to_mp, mp_dist, sc_dist, sza

### Plot distance outside magnetopause, spacecraft and magnetopause locations, and solar zenith angle

Let's plot each for all five of our time ranges. It will take about 3 minutes.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline


n = len(tranges)
fig1, axs1 = plt.subplots(1, n, figsize=(n*5, 2.5), sharey=True)
fig2, axs2 = plt.subplots(1, n, figsize=(n*5, 2.5), sharey=True)
fig3, axs3 = plt.subplots(1, n, figsize=(n*5, 2.5), sharey=True)
fig4, axs4 = plt.subplots(1, n, figsize=(n*5, 2.5), sharey=True)

for i, trange in enumerate(tranges):
    mec_vars, fgm_vars, ion_vars = load_mms_data(trange)
    epoch, pos, sc_to_mp, mp_dist, sc_dist, sza = spacecraft_magnetopause_calculations(mec_vars)

    # Spacecraft distance outside magnetopause
    fig1.suptitle('Distance Outside Magnetopause')
    axs1[i].plot(epoch, sc_to_mp)
    axs1[i].set(xlabel='Time', ylabel='Re')
    fig1.autofmt_xdate()

    # Spacecraft and magnetopause locations
    fig2.suptitle('Distance From Earth')
    l_mp, = axs2[i].plot(epoch, mp_dist, label='Magnetopause')
    l_sc, = axs2[i].plot(epoch, sc_dist, label='Spacecraft')
    axs2[i].set(xlabel='Time', ylabel='Re')
    fig2.autofmt_xdate()

    # Solar zenith angle
    # Identifying cause of weird bump in first time range..it's crossing to the nightside
    fig3.suptitle('Solar Zenith Angle')
    axs3[i].plot(epoch, numpy.degrees(sza))
    axs3[i].set(xlabel='Time')
    fig3.autofmt_xdate()

    # More context of the nightside crossing
    fig4.suptitle('Spacecraft Coordinates')
    l_x, = axs4[i].plot(epoch, pos[:, 0] / 6378, label='x')
    l_y, = axs4[i].plot(epoch, pos[:, 1] / 6378, label='y')
    l_z, = axs4[i].plot(epoch, pos[:, 2] / 6378, label='z')
    fig4.autofmt_xdate()

fig2.legend([l_mp, l_sc], [l_mp.get_label(), l_sc.get_label()], loc='right')
fig4.legend([l_x, l_y, l_z], [l_x.get_label(), l_y.get_label(), l_z.get_label()], loc='right')


#### Plot MMS data to visualize

In [ ]:
tplot(['mms1_dis_energyspectr_omni_fast', 'mms1_dis_numberdensity_fast', 'mms1_fgm_b_gsm_srvy_l2_bvec', 'mms1_mec_r_gsm'])  # Plot a multi-instrument figure

## Prepare data for PlasmaPy:

Here we show how to calculate various plasma parameters (Alfvén speed, plasma beta, Debye length, etc.) from our MMS data and the PlasmaPy package.

#### Interpolate to a common set of times
Now we need to interpolate the B-field and DES (electron) data to the DIS (ion) time stamps.

Note: `tinterpol` creates a new variable containing the interpolated output with the suffix '-itrp'.

In [ ]:
from pyspedas import tinterpol
tinterpol('mms1_fgm_b_gse_srvy_l2_btot', 'mms1_dis_numberdensity_fast')
tinterpol('mms1_des_numberdensity_fast', 'mms1_dis_numberdensity_fast')
tinterpol('mms1_des_temppara_fast', 'mms1_dis_numberdensity_fast')
tinterpol('mms1_des_tempperp_fast', 'mms1_dis_numberdensity_fast')

#### Extract the data values

In [ ]:
from pytplot import get_data
fgm_b = get_data('mms1_fgm_b_gse_srvy_l2_btot-itrp')
dis_n = get_data('mms1_dis_numberdensity_fast')
dis_Tpara = get_data('mms1_dis_temppara_fast')
dis_Tperp = get_data('mms1_dis_tempperp_fast')
des_n = get_data('mms1_des_numberdensity_fast-itrp')
des_Tpara = get_data('mms1_des_temppara_fast-itrp')
des_Tperp = get_data('mms1_des_tempperp_fast-itrp')

#### Calculate T from Tpara and Tperp

Temperature data released by the FPI team come as parallel and perpendicular components; to calculate plasma parameters, we'll need the total temperature.

For details on this calculation, please see the FPI Data Product Guide:
https://lasp.colorado.edu/galaxy/pages/viewpage.action?pageId=37618954

In [ ]:
dis_T = (dis_Tpara.y + 2*dis_Tperp.y)/3.0
des_T = (des_Tpara.y + 2*des_Tperp.y)/3.0

#### Add units to the data

PlasmaPy requires us to specify the units of the data using `astropy` units

In [ ]:
from astropy import units as u
B = fgm_b.y * u.nT
n_i = dis_n.y * u.cm**-3
n_e = des_n.y * u.cm**-3
T_i = dis_T * u.eV
T_e = des_T * u.eV

Now that we have some data loaded, we'll calculate some plasma parameters.

## Calculate plasma parameters:

In [ ]:
import plasmapy

#### Alfvén speed

In [ ]:
Va = plasmapy.formulary.parameters.Alfven_speed(B, n_i, 'p')

# convert to km / s
Va = Va.to(u.km / u.s)

#### Plasma beta

In [ ]:
# ions
beta_i = plasmapy.formulary.dimensionless.beta(T_i, n_i, B)

# electrons
beta_e = plasmapy.formulary.dimensionless.beta(T_e, n_e, B)

# combined
beta = beta_i + beta_e

#### Ion inertial length

In [ ]:
d_i = plasmapy.formulary.parameters.inertial_length(n_i, 'p+')

#### Debye length

In [ ]:
lamda_d = plasmapy.formulary.parameters.Debye_length(T_e, n_e)

#### Ion gyrofrequency


In [ ]:
omega_ci = plasmapy.formulary.parameters.gyrofrequency(B, 'p', to_hz=True)

#### Ion gyroradius

In [ ]:
# r_i = plasmapy.formulary.parameters.gyroradius(B, 'p', T_i=T_i)

# convert to km
# r_i = r_i.to(u.km)

#### Bohm diffusion coefficient

In [ ]:
DB = plasmapy.formulary.parameters.Bohm_diffusion(T_e, B)

#### Lower hybrid frequency

In [ ]:
omega_lh = plasmapy.formulary.parameters.lower_hybrid_frequency(B, n_i, 'p', to_hz=True)

#### Upper hybrid frequency

In [ ]:
omega_uh = plasmapy.formulary.parameters.upper_hybrid_frequency(B, n_e, to_hz=True)

## Plotting
Now that we calculated the plasma parameters, let's move on to plotting.

#### Save the data in tplot variables

In [ ]:
from pytplot import store_data
store_data('alfven_speed', data={'x': fgm_b.times, 'y': Va})
store_data('plasma_beta', data={'x': fgm_b.times, 'y': beta})
store_data('ion_inertial_length', data={'x': fgm_b.times, 'y': d_i})
store_data('debye_length', data={'x': fgm_b.times, 'y': lamda_d})
store_data('omega_ci', data={'x': fgm_b.times, 'y': omega_ci})
# store_data('ion_gyroradius', data={'x': fgm_b.times, 'y': r_i})
store_data('bohm_diffusion_coeff', data={'x': fgm_b.times, 'y': DB})
store_data('omega_lh', data={'x': fgm_b.times, 'y': omega_lh})
store_data('omega_uh', data={'x': fgm_b.times, 'y': omega_uh})

#### Set some plot metadata

In [ ]:
from pytplot import options
options('alfven_speed', 'ytitle', 'Va \\ (' + str(Va.unit) + ')')
options('alfven_speed', 'legend_names', 'Alfvén speed')
options('plasma_beta', 'ytitle', 'Beta')
options('plasma_beta', 'legend_names', 'Plasma Beta')
options('ion_inertial_length', 'ytitle', 'd_i \\ (' + str(d_i.unit) + ')')
options('ion_inertial_length', 'legend_names', 'Ion inertial length')
options('debye_length', 'ytitle', 'Lamda_d \\ (' + str(lamda_d.unit) + ')')
options('debye_length', 'legend_names', 'Debye length')
options('omega_ci', 'ytitle', 'omega_ci \\ (' + str(omega_ci.unit) + ')')
options('omega_ci', 'legend_names', 'Ion gyrofrequency')
# options('ion_gyroradius', 'ytitle', 'r_i \\ (' + str(r_i.unit) + ')')
# options('ion_gyroradius', 'legend_names', 'Ion gyroradius')
options('bohm_diffusion_coeff', 'ytitle', 'DB \\ (' + str(DB.unit) + ')')
options('bohm_diffusion_coeff', 'legend_names', 'Bohm diffusion coefficient')
options('omega_uh', 'ytitle', 'omega_uh \\ (' + str(omega_uh.unit) + ')')
options('omega_uh', 'legend_names', 'Upper hybrid frequency')
options('omega_lh', 'ytitle', 'omega_lh \\ (' + str(omega_lh.unit) + ')')
options('omega_lh', 'legend_names', 'Lower hybrid frequency')

## Plot the plasma parameters and compare:

In [ ]:
tplot(['alfven_speed', 
       'plasma_beta',
       'ion_inertial_length',
       'debye_length',
       'omega_ci',
       # 'ion_gyroradius',
       'bohm_diffusion_coeff',
       'omega_uh',
       'omega_lh'])

Let's compare that to our MMS data and model data one more time:

In [ ]:
# MMS data
tplot(['mms1_dis_energyspectr_omni_fast', 'mms1_dis_numberdensity_fast', 'mms1_fgm_b_gsm_srvy_l2_bvec', 'mms1_mec_r_gsm'])

# MMS and magnetopause locations
fig2, axs2 = plt.subplots(1, 1, figsize=(8, 4), sharey=True)
fig2.suptitle('Distance From Earth')
l_mp, = axs2.plot(epoch, mp_dist, label='Magnetopause')
l_sc, = axs2.plot(epoch, sc_dist, label='Spacecraft')
axs2.set(xlabel='Time', ylabel='Re')
fig2.autofmt_xdate()
fig2.legend([l_mp, l_sc], [l_mp.get_label(), l_sc.get_label()], loc='right')

Note how you can visually inspect the real-world plots to see they all show dramatic changes between 00:30 – 00:40. This disagrees with the model that predicts the crossing would've happened just after 00:40, **so we can conclude the magnetopause was closer to Earth than the model predicted.**

## Conclusion

We can use the five PyHC software packages pySPEDAS, SpacePy, PlasmaPy, PyTplot, and Kamodo to detect magnetopause boundary crossings in MMS data. Then we can compare where these crossings happen to the locations estimated by models of the magnetopause to test their accuracy. We've shown at least one example where the magnetopause was closer to the Earth than predicted by the Shue model. This is not entirely surprising becuase models are just that—only models. But the novel workflow we developed here showcases how PyHC's packages can simplify this kind of analysis. We hope our field takes this as an example of how scientists and software developers can work together to produce high-quality research that is both transparent and reproducible.

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=101b9646-3fd0-4978-a48e-a4f3e708a0ac' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>